In this notebook I walk through the generation/inferece pipeline (spending most of the time on inference).

In [1]:
from abc import ABC, abstractmethod

import os
from tqdm import tqdm
import math 
import time

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from scipy import stats

import torch
from torch.distributions import Beta
from torch.distributions.bernoulli import Bernoulli
from torch.nn.functional import log_softmax
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader, TensorDataset

In [2]:
SEED = 1
np.random.seed(SEED)
torch.manual_seed(SEED)

### Generation:

Here I am using 2/10 generation.

In [3]:
### these params control the generation scheme
rho        = 0.8   # polarization
pop_size   = 50000 # num individuals
epsilon    = 0.05  # expected prop of speech consisting of neutral words
pi         = 0.5   # pi == 0.5 => beta mixture symmetrical (choose beta1 with prob pi = 0.5)
speech_len = 15    # words per speech

In [44]:
def generate(rho=rho, N=pop_size, epsilon=epsilon, pi=pi, speech_len=speech_len, verbose=True):
    """
    Uses 2/10 generation scheme to generate N samples.

    Returns:
        (X, y), (a, b, rho, epsilon, u)
        X.size() == [N, 3] is a vector of word counts
        y.size() == [N] is a vector of political parties
        u.shape  == (N,) is a vector of individual stances
        rho is true polarization
        epsilon is expected prop of neutral words
        a, b are true alpha/beta for beta mixture model
    """
    start = time.time()
    if verbose:
        print(f'Beginning Data Generation...')
        print(f'=' * 20)
        
    ### get beta mixture model params 
    sigma = 0.175 * (rho ** 2) - 0.3625 * rho + 0.1875
    a     = rho * ((rho * (1 - rho)) / sigma - 1)
    b     = (1 - rho) * ((rho * (1 - rho)) / sigma - 1)

    if verbose:
        print(f"True Alpha: {a}")
        print(f"True Beta: {b}")
        print(f"True Epsilon: {epsilon}\n")

    mean = a / (a + b)
    var  = a * b / ((a + b)**2 * (a + b + 1))

    if abs(mean - rho) > 10e-15:
        print(f'Mean: {mean}')
        print(f"Rho: {rho}")
        raise AssertionError(f"Mean of BMM params should be rho")

    if abs(var - sigma) > 10e-15:
        print(f'Var: {var}')
        print(f'Sigma: {sigma}')
        raise AssertionError(f"Var of BMM params should be sigma")

    ### u ~ pi Beta(a, b) + (1 - pi) Beta(b, a)
    weights = [pi, 1-pi]
    mixture_samples = np.random.choice([0, 1], size=N, p=weights)

    u = 2 * np.where(mixture_samples == 1, stats.beta.rvs(a, b, size=N), stats.beta.rvs(b, a, size=N)) - 1

    if u.shape != (N,):
        raise AssertionError(f"u.shape should be (N,)")

    if verbose:
        print(f'mixture samples: {mixture_samples[:5]}')
        print(f'u samples: {u[:5]}\n')

    ### y = 1(u >= 0)
    y = (u >= 0).astype(int)

    if verbose:
        print(f'y samples: {y[:5]}\n')

    ### phi is a prob matrix that is a function of u, epsilon
    phi = np.array([(1 - (u+1)/2) * (1 - epsilon), (u+1)/2 * (1 - epsilon), np.repeat(epsilon, N)]).T
    
    if verbose:
        print(f'phi samples:\n {phi[:5, :]}\n')

    if phi.shape != (N, 3):
        raise AssertionError(f'phi.shape should be (N, V) == (N, 3)')

    if abs(sum(phi[0]) - 1) > 10e-5:
        raise AssertionError(f'rows of phi should sum to 1')
    
    X = np.array([stats.multinomial.rvs(n=speech_len, p=phi[i, :]) for i in range(N)])

    if verbose:
        print(f'X samples:\n {X[:5, :]}\n')
    
    if X.shape != (N, 3):
        raise AssertionError(f'X.shape should be (N, V) == (N, 3)')

    if X[:5].sum() != 15 * 5:
        raise AssertionError(f'rows of phi should sum to 1')

    X = torch.from_numpy(X).to(torch.float32)
    y = torch.from_numpy(y).to(torch.float32)

    known   = (X, y)
    unknown = (a, b, rho, epsilon, u)

    if verbose:
        print('=' * 20)
        print(f'Generation Time: {round(time.time() - start, 3)} seconds for {N} samples.')

    return known, unknown

known, unknown = generate()
X, y = known
a, b, rho, epsilon, u = unknown

Beginning Data Generation...
True Alpha: 2.0000000000000004
True Beta: 2.0000000000000004
True Epsilon: 0.05

mixture samples: [1 1 0 1 1]
u samples: [-0.29375126 -0.11814876 -0.29827691 -0.07967257 -0.64165267]

y samples: [0 0 0 0 0]

phi samples:
 [[0.61453185 0.33546815 0.05      ]
 [0.53112066 0.41887934 0.05      ]
 [0.61668153 0.33331847 0.05      ]
 [0.51284447 0.43715553 0.05      ]
 [0.77978502 0.17021498 0.05      ]]

X samples:
 [[11  3  1]
 [ 8  7  0]
 [10  3  2]
 [ 8  7  0]
 [14  1  0]]

Generation Time: 2.969 seconds for 50000 samples.


### Inference

The goal of inference is to recover $u$ given a sample of $(x,y)$ pairs. We assume the following data generating process:
\begin{align*}
    u &\sim 2 \cdot \left(\pi\cdot\mathrm{Beta}\left(u;\alpha,\beta\right)+(1-\pi)\cdot\mathrm{Beta}\left(u;\beta,\alpha\right)\right)-1 \\
    y &= 1(u \ge \lambda) \\
    x &\sim \mathrm{Multinomial}\left(S, \mathrm{Softmax}(Wu)\right), W \in \R^V.
\end{align*}

For now, we assume $\lambda = 0, V = 3, \pi = 0.5$, so our parameters are $\theta = \{\alpha, \beta, W\}$. Given we assume this full form of $u$ parameterized by $\alpha, \beta$, recovering the distribution of $u$ is equivalent to recovering $\alpha, \beta$. This also gives us the following joint:

\begin{align*}

p(u,x^{(n)},y^{(n)};\theta) &= p(u)p(y^{(n)}|u)\prod_{s=1}^{S}p(x^{(n)}_s | u) \\
&= \left(\frac{1}{4}\mathrm{Beta}\left(\frac{u+1}{2};\alpha,\beta\right)+\frac{1}{4}\mathrm{Beta}\left(\frac{u+1}{2};\beta,\alpha\right)\right) \\&\cdot 1\left(1(u\ge0)=y^{(n)}\right)\frac{S!}{x^{(n)}_1!x^{(n)}_2!x^{(n)}_3!}\prod_{s=1}^{S}\mathrm{softmax}(Wu)_{x^{(n)}_s}

\end{align*}

Here, we can get the density of $u$ by noting $u$ is a monotonic transform of a random variable whose density is known. Thus, applying a formula found [here](https://en.wikipedia.org/wiki/Probability_density_function#Function_of_random_variables_and_change_of_variables_in_the_probability_density_function) we get the above expression.

The basic idea is to maximize the log marginal likelihood. Let $\theta = \{\alpha, \beta, W\}$. We'd like to find 
$$
\theta^* = \argmax_{\theta} \underbrace{\log p(x^{(1:N)}, y^{(1:N)}; \theta)}_{\mathcal{L}(\theta)}. 
$$


The goal is to recover $\alpha, \beta$ (hence $u$ given we are assuming $u \sim 2\left(\frac{1}{2} \cdot \mathrm{Beta}(\alpha, \beta) + \frac{1}{2} \cdot \mathrm{Beta}(\beta, \alpha)\right) - 1$ given $(x,y)$ pairs.

We cannot directly compute this argmax. So, to get MLE estimates for alpha, beta, W, we will take gradient descent steps on the log marginal likelihood. Its gradient is given by the following expression:
$$
\nabla_\theta \log p(x^{(1:N)},y^{(1:N)}) = \sum_{n=1}^{N} \mathbb{E}_{p(u|x^{(n)},y^{(n)};\theta)}[\nabla_\theta \log p(u,x^{(n)},y^{(n)};\theta)].
$$
There are a number of issues here, beginning with the fact that we cannot analytically recover the posterior due to the integral being intractable. However, since $u \in [-1,1]$, we can discretize the integral, which will be simpler than something like MCMC for sampling/using variational inference to approximate the posterior. To do this, let's write the definition of the expectation:
$$
\nabla_\theta \log p(x^{(1:N)},y^{(1:N)}) = \sum_{n=1}^{N} \int_{-1}^{1} \frac{p(x^{(n)}, y^{(n)}, u;\ \theta)}{p(x^{(n)},y^{(n)};\  \theta)}\nabla_\theta \log p(u,x^{(n)},y^{(n)};\theta) \mathrm{d}u
$$
Now, we could assume $u$ takes on $G$ linearly spaced points in $[-1,1]$. One way to do this is to essentially keep track of a hashmap $u \mapsto p(u, x^{(n)}, y^{(n)})$: 
$$
\text{Example:}\  \{u = -.99 : 0.03, \dots, u = .99 : 0.007\}.
$$
The sum of the values of this hashmap, by the law of total probability, is $p(x^{(n)}, y^{(n)})$. So, the posterior can be obtained by normalizing the values of the hashmap by the sum of the values. However, we can do better. Notice that when $y^{(n)} \ne 1(u \ge 0)$, $p(u, x^{(n)}, y^{(n)})$ is necessarily $0$. In other words, half of the entries of our discrete distribution will be $0$ in a very predictable way. Thus, we can just ignore these values and use $y^{(n)}$ as a condition. 

If $y^{(n)} = 1$, have $u$ take on $G$ linearly spaced points in $[0,1]$, and if $y^{(n)} = 0$ do $[-1,0]$. This is what we do in practice. Theoretically, we're still computing:
$$
\nabla_\theta \log p(x^{(1:N)},y^{(1:N)}) = \sum_{n=1}^{N} \sum_{u} \frac{p(x^{(n)}, y^{(n)}, u;\ \theta)}{p(x^{(n)},y^{(n)};\  \theta)}\nabla_\theta \log p(u,x^{(n)},y^{(n)};\theta).
$$
It's convenient to move the gradient outside the sum, since then we can efficiently compute the double sum via matrices and then just sum the matrix/call .backward(). This is what I was doing initially, but it's definitely wrong, since the posterior relies on theta. To fix this, I call .detach() on the "weight" (posterior probability), which means the gradients only flow through the joint log probability (which is what we want).

In [53]:
grid_size  = 200 ### discretize integral into grid_size linearly spaced pts
batch_size = 100 ### num samples processed simultaneously
num_epochs = 200 ### number epochs
lr         = 0.1 ### learning rate

def compute_nll(x_batch, y_batch, log_alpha, log_beta, W):
    """
    Parameters:
        (x_batch, y_batch) a batch of B (x,y) samples 
        (log_alpha, log_beta, W) trainable parameters

    Returns:
        A scaler quantity s.t. calling .backward() will compute
        grad_{theta} (L(theta))
    """
    raise NotImplementedError

def train(X, y, num_epochs=num_epochs, batch_size=batch_size, grid_size=grid_size, lr=lr, verbose=True):
    if verbose:
        print(f'Beginning Inference...')
        print(f'=' * 20)
        print(f'Hyperparameters:')
        print(f'Dataset Length: {len(X)}')
        print(f'Number Epochs: {num_epochs}')
        print(f'Batch Size: {batch_size}')
        print(f'Grid Size: {grid_size}')
        print(f"Learning Rate: {lr}")
        print('=' * 20)

    ### not strictly necessary 
    assert len(X) % batch_size == 0
    
    start = time.time()

    dataset    = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    X_batch, y_batch = list(dataloader)[0]

    if list(X_batch.size()) != [batch_size, 3]:
        raise AssertionError(f"X_ex.size() should be [batch_size, V] == [batch_size, 3], got {X_batch.size()}")

    if list(y_batch.size()) != [batch_size]:
        raise AssertionError(f"y_ex.size() should be [batch_size], got {y_batch.size()}")

    ### play with parameter initialization!
    log_alpha = torch.normal(2, 1, size=(1,), requires_grad=True)
    log_beta  = torch.normal(0, 1, size=(1,), requires_grad=True)
    W         = torch.normal(0, 2, size=(3,), requires_grad=True)

    if verbose:
        print(f'Initial Parameters:')
        print(f'Alpha: {torch.exp(log_alpha).item()}')
        print(f"Beta: {torch.exp(log_beta).item()}")
        print(f"W: {W.data.tolist()}")

    optimizer = SGD([log_alpha, log_beta, W], lr=lr)
    
    for epoch in tqdm(range(num_epochs)):
        if verbose:
            print(f'Epoch {epoch+1} of {num_epochs}')
        ### only step after each epoch
        optimizer.zero_grad() 
        nll = 0
        for x_batch, y_batch in dataloader:
            nll += compute_nll(x_batch, y_batch, log_alpha, log_beta, W, grid_size, batch_size)
        
        nll = nll / len(dataloader.dataset)
        nll.backward()
        optimizer.step()
        if verbose:
            print(f'Epoch {epoch+1}, NLL: {round(nll.item(), 3)}, Alpha: {round(np.exp(log_alpha.item()),3)}, Beta: {round(np.exp(log_beta.item()),3)}, W: {W.tolist()}')
            print('=' * 20)

    final_alpha = np.exp(log_alpha.item())
    final_beta  = np.exp(log_beta.item())

    if verbose:
        print(f"Training Took {round(time.time() - start, 2)} Seconds.")
        
    print(f"Trained Params: alpha: {final_alpha}, beta: {final_beta}, W: {W.tolist()}")

    return final_alpha, final_beta, W

In [54]:
### numerically stable factorial
factorial = lambda x : torch.exp(torch.lgamma(x+1))

assert factorial(torch.tensor(3.)) == torch.tensor(6.)
print(f"3.0001! = {factorial(torch.tensor(3.001)).item()}")

### notice this is infinity, so we can't call factorial on negative stuff
print(f"-1! = {factorial(torch.tensor(-1.)).item()}")

def compute_log_joint(u_mat, x_batch, y_batch, log_alpha, log_beta, W, grid_size, batch_size):
    """
    Parameters:
        x_batch (size [batch_size, 3])
        y_batch (size [batch_size])
        u_mat   (size [batch_size, grid_size])
        (log_alpha, log_beta, W) trainable params 
    
    Returns:
        torch.tensor (size [batch_size, grid_size]), where the (i,j)th element 
        is the log joint probability (log p(u, x, y; theta)). Here u is the (i,j)th
        element of u_mat, x is the ith row of x_batch, y is the ith element of y_batch

    """
    assert list(u_mat.size())   == [batch_size, grid_size] 
    assert list(x_batch.size()) == [batch_size, 3]
    assert list(y_batch.size()) == [batch_size]

    if not torch.all((u_mat >= 0).float() == y_batch.unsqueeze(1)).item():
        raise ValueError(f"u_mat is incompatible with y_batch.")

    ### log prior: log p(u) = log(1/4 Beta((u+1)/2; a, b) + 1/4 Beta((u+1)/2; b, a))
    ###                     = log(1/4) + log(Beta((u+1)/2; a, b) + Beta((u+1)/2; b, a))
    alpha = torch.exp(log_alpha)
    beta  = torch.exp(log_beta)
        
    beta_dist_ab = Beta(alpha, beta)
    beta_dist_ba = Beta(beta, alpha)

    ### see 3-6 testing.ipynb test1 to see how log_prob broadcasts
    log_beta_prob_ab = beta_dist_ab.log_prob((u_mat + 1) / 2)
    log_beta_prob_ba = beta_dist_ba.log_prob((u_mat + 1) / 2)

    prior = torch.log(torch.exp(log_beta_prob_ab) + torch.exp(log_beta_prob_ba)) + torch.log(torch.tensor(0.25))

    assert list(prior.size()) == [batch_size, grid_size]
  
    ### log likelihood: log p(x, y | u) = dot(x, log Softmax(Wu)) + log(S!/(x1! x2! x3!)) 

    ### step 1: compute Wu (see 3-6 testing.ipynb test2 to see this logic)
    W_expanded = W.unsqueeze(0).unsqueeze(0)
    u_expanded = u_mat.unsqueeze(2)
    Wu = torch.matmul(u_expanded, W_expanded) 

    assert list(W_expanded.size()) == [1,1,3]
    assert list(u_expanded.size()) == [batch_size, grid_size, 1]
    assert list(Wu.size()) == [batch_size, grid_size, 3]

    ### step 2: dot(x, log Softmax(Wu)) (see 3-6 testing.ipynb test3 to see this logic)
    likelihood = (x_batch.unsqueeze(1) * log_softmax(Wu, dim=2)).sum(dim=2)

    ### ensure not taking negative factorials
    assert torch.all(x_batch >= 0)

    ### step 3: multinomial constant factors (see 3-6 testing.ipynb test4)
    likelihood += torch.log(factorial(torch.tensor(speech_len)))
    likelihood -= torch.log(factorial(x_batch)).sum(dim=1, keepdim=True) 

    assert list(likelihood.size()) == [batch_size, grid_size]
    return prior + likelihood


def compute_nll(x_batch, y_batch, log_alpha, log_beta, W, grid_size, batch_size):
    """
    Parameters:
        x_batch (size [batch_size, 3])
        y_batch (size [batch_size])
        log_alpha, log_beta, W (trainable params)
        grid_size (int) number of lin spaced points to discretize integral into
    
    Returns:
        quantity s.t. .backward() would be the gradient of the nll
    """
    ### see 3-6 testing.ipynb test5 to see this logic
    u_mat = torch.empty(batch_size, grid_size)
    u_mat[y_batch == 1] = torch.linspace(1/(grid_size+1), 1-1/(grid_size+1), grid_size).repeat((y_batch == 1).sum(), 1)
    u_mat[y_batch == 0] = torch.linspace(-1+1/(grid_size+1), -1/(grid_size+1), grid_size).repeat((y_batch == 0).sum(), 1)

    assert list(u_mat.size()) == [batch_size, grid_size]

    log_joint = compute_log_joint(u_mat, x_batch, y_batch, log_alpha, log_beta, W, grid_size, batch_size)

    assert list(log_joint.size()) == [batch_size, grid_size]

    ### see 3-6 testing.ipynb test6 to see this logic
    max_log_prob = torch.max(log_joint, dim=1, keepdim=True)[0]
    joint_probs = torch.exp(log_joint - max_log_prob)
    posterior = joint_probs / joint_probs.sum(dim=1, keepdim=True)  

    # detach posterior (see 3-6 testing.ipynb test7 to verify this works)
    weighted_log_joint = posterior.detach() * log_joint
    nll = -weighted_log_joint.sum()

    return nll

3.0001! = 6.007541656494141
-1! = inf


In [55]:
for rho in [0.5, 0.6, 0.7, 0.8, 0.9]:
    print(f'=' * 20)
    print(f'Testing Inference on Rho = {rho}')
    known, unknown = generate(rho=rho, N=10000, verbose=False)
    X, y = known
    a, b, rho, epsilon, u = unknown

    trained_a, trained_b, trained_W = train(X, y, batch_size = 10000, verbose=False)
    print(f'Rho Estimate: {(max(trained_a, trained_b))/(trained_a + trained_b)}')

Testing Inference on Rho = 0.5


100%|██████████| 200/200 [00:35<00:00,  5.63it/s]


Trained Params: alpha: 6.8232979285514315, beta: 4.522689660344638, W: [-2.763429641723633, 1.848551630973816, -0.5217438340187073]
Rho Estimate: 0.6013842228444848
Testing Inference on Rho = 0.6


100%|██████████| 200/200 [00:40<00:00,  4.98it/s]


Trained Params: alpha: 1.7179977443541095, beta: 1.399244604814195, W: [-2.4644391536712646, 0.4405503571033478, -1.0120877027511597]
Rho Estimate: 0.5511274235102316
Testing Inference on Rho = 0.7


...

Trained Params: alpha: 2.170460373265954, beta: 1.4488401813836178, W: [-2.433741331100464, 1.1385084390640259, -0.6374872922897339]
Rho Estimate: 0.5996905591268594
Testing Inference on Rho = 0.8


100%|██████████| 200/200 [00:35<00:00,  5.57it/s]


Trained Params: alpha: 4.262234093831484, beta: 1.2476877443481504, W: [-1.943561315536499, 1.802489161491394, -0.06956668943166733]
Rho Estimate: 0.7735561808331639
Testing Inference on Rho = 0.9


100%|██████████| 200/200 [00:37<00:00,  5.32it/s]

Trained Params: alpha: 13.799174273394556, beta: 3.262731913179484, W: [-1.6382172107696533, 3.7558364868164062, 1.0461071729660034]
Rho Estimate: 0.8087709616087962
